In [11]:
import pandas as pd
import cobra
import numpy as np
import matplotlib.pyplot as plt

# Community modeling

In this notebook we will implement a method to create community models of two or more species specific metabolic models using cobrapy.

In [12]:
model_DP = cobra.io.read_sbml_model("models/consistent_DP_SNM.xml")
model_SA = cobra.io.read_sbml_model("models/consistent_iYS854_SNM.xml")
print("Growth: ", model_DP.slim_optimize())
print("Growth: ", model_SA.slim_optimize())

Growth:  0.2823653925322476
Growth:  2.558694612613393


In [13]:
snm3 = pd.read_csv("SNM3.csv", sep =";")
snm3

,Compound,BiGG,ModelSeed,KEGG
0,Alanine,ala__L,cpd00035,C00041
1,Arginine,arg__L,cpd00051,C00062
2,Cysteine,cys__L,cpd00084,C00097
3,Glutamic acid,glu__L,cpd00023,C00025
4,Glycine,gly,cpd00033,C00037
5,Histidine,his__L,cpd00119,C00135
6,Leucine,leu__L,cpd00107,C00123
7,Lysine,lys__L,cpd00039,C00047
8,Ornithine,orn,cpd00064,C00077
9,Phenylalanine,phe__L,cpd00066,C00079


In [14]:
BIOMASS_DP = "Growth" 
BIOMASS_SA = "BIOMASS_iYS_wild_type"
models = [model_DP.copy(), model_SA.copy()]

## 2) Bag of reactions community model

Before we created compartments to seperate both organisms. Now we will only consider a bag of reactions, throwing all reactions into one bucket.

In [15]:
def create_bag_of_react_model(models, biomass_functions, weights="auto"):
    # Bag of words model
    model = model_DP + model_SA

    if weights == "auto":
        biomass = []
        for model_i in models:
            biomass.append(model_i.slim_optimize())
        max_biomass = np.max(biomass)
        weights = [max_biomass/g for g in biomass]
    
    # Set objective as sum of growth
    objective = []
    for i, growth in enumerate(biomass_functions):
        v_i = weights[i] * model.reactions.get_by_id(growth).flux_expression
        objective.append(v_i)
    model.objective = sum(objective)
    return model

In [16]:
model1_1 = create_bag_of_react_model([model_DP,model_SA],[BIOMASS_DP, BIOMASS_SA], weights=[1.,1.])

Ignoring reaction 'ACALD' since it already exists.
Ignoring reaction 'ACALDt' since it already exists.
Ignoring reaction '3OAS140' since it already exists.
Ignoring reaction '3OAS60' since it already exists.
Ignoring reaction 'ACCOAC' since it already exists.
Ignoring reaction '5DOAN' since it already exists.
Ignoring reaction '6PHBG' since it already exists.
Ignoring reaction '6PHBG2' since it already exists.
Ignoring reaction 'ACGS' since it already exists.
Ignoring reaction 'ACKr' since it already exists.
Ignoring reaction '3HAD100' since it already exists.
Ignoring reaction '3HAD120' since it already exists.
Ignoring reaction '3HAD140' since it already exists.
Ignoring reaction 'ACACT1r' since it already exists.
Ignoring reaction '3HAD60' since it already exists.
Ignoring reaction 'ACM6PH' since it already exists.
Ignoring reaction '3OAR100' since it already exists.
Ignoring reaction '3OAR140' since it already exists.
Ignoring reaction '3OAR60' since it already exists.
Ignoring rea

Ignoring reaction 'MOHMT' since it already exists.
Ignoring reaction 'MTHFC' since it already exists.
Ignoring reaction 'MTHFD' since it already exists.
Ignoring reaction 'MCOATA' since it already exists.
Ignoring reaction 'ME1' since it already exists.
Ignoring reaction 'NACt' since it already exists.
Ignoring reaction 'ME2' since it already exists.
Ignoring reaction 'METAT' since it already exists.
Ignoring reaction 'METS' since it already exists.
Ignoring reaction 'NAt3_1' since it already exists.
Ignoring reaction 'METabc' since it already exists.
Ignoring reaction 'NDPK2' since it already exists.
Ignoring reaction 'MEVK2' since it already exists.
Ignoring reaction 'MEVK3' since it already exists.
Ignoring reaction 'MEVK4' since it already exists.
Ignoring reaction 'NDPK4' since it already exists.
Ignoring reaction 'NNAM' since it already exists.
Ignoring reaction 'NNATr' since it already exists.
Ignoring reaction 'GMPR' since it already exists.
Ignoring reaction 'GMPS2' since it a

In [17]:
print("MBR Weights 1:1: ", model1_1.slim_optimize())
print("SA growth: " + str(model1_1.reactions.get_by_id(BIOMASS_SA).flux))
print("DP growth: " + str(model1_1.reactions.get_by_id(BIOMASS_DP).flux))

MBR Weights 1:1:  5.359696176521698
SA growth: 5.010482140405494
DP growth: 0.3492140361162044


In [18]:
opt = model1_1.optimize()
print("SA growth: " + str(model1_1.reactions.get_by_id(BIOMASS_SA).flux))
print("DP growth: " + str(model1_1.reactions.get_by_id(BIOMASS_DP).flux))

SA growth: 5.010482140405494
DP growth: 0.3492140361162044


In [19]:
model10_1 = create_bag_of_react_model([model_DP,model_SA],[BIOMASS_DP, BIOMASS_SA], weights=[10.,1.])

Ignoring reaction 'ACALD' since it already exists.
Ignoring reaction 'ACALDt' since it already exists.
Ignoring reaction '3OAS140' since it already exists.
Ignoring reaction '3OAS60' since it already exists.
Ignoring reaction 'ACCOAC' since it already exists.
Ignoring reaction '5DOAN' since it already exists.
Ignoring reaction '6PHBG' since it already exists.
Ignoring reaction '6PHBG2' since it already exists.
Ignoring reaction 'ACGS' since it already exists.
Ignoring reaction 'ACKr' since it already exists.
Ignoring reaction '3HAD100' since it already exists.
Ignoring reaction '3HAD120' since it already exists.
Ignoring reaction '3HAD140' since it already exists.
Ignoring reaction 'ACACT1r' since it already exists.
Ignoring reaction '3HAD60' since it already exists.
Ignoring reaction 'ACM6PH' since it already exists.
Ignoring reaction '3OAR100' since it already exists.
Ignoring reaction '3OAR140' since it already exists.
Ignoring reaction '3OAR60' since it already exists.
Ignoring rea

Ignoring reaction 'MOHMT' since it already exists.
Ignoring reaction 'MTHFC' since it already exists.
Ignoring reaction 'MTHFD' since it already exists.
Ignoring reaction 'MCOATA' since it already exists.
Ignoring reaction 'ME1' since it already exists.
Ignoring reaction 'NACt' since it already exists.
Ignoring reaction 'ME2' since it already exists.
Ignoring reaction 'METAT' since it already exists.
Ignoring reaction 'METS' since it already exists.
Ignoring reaction 'NAt3_1' since it already exists.
Ignoring reaction 'METabc' since it already exists.
Ignoring reaction 'NDPK2' since it already exists.
Ignoring reaction 'MEVK2' since it already exists.
Ignoring reaction 'MEVK3' since it already exists.
Ignoring reaction 'MEVK4' since it already exists.
Ignoring reaction 'NDPK4' since it already exists.
Ignoring reaction 'NNAM' since it already exists.
Ignoring reaction 'NNATr' since it already exists.
Ignoring reaction 'GMPR' since it already exists.
Ignoring reaction 'GMPS2' since it a

In [20]:
opt = model10_1.optimize()
print("SA growth: " + str(model10_1.reactions.get_by_id(BIOMASS_SA).flux))
print("DP growth: " + str(model10_1.reactions.get_by_id(BIOMASS_DP).flux))

SA growth: 0.0
DP growth: 1.3402800000000015


In [21]:
print("MBR Weights 10:1: ", model10_1.slim_optimize())
print("SA growth: " + str(model10_1.reactions.get_by_id(BIOMASS_SA).flux))
print("DP growth: " + str(model10_1.reactions.get_by_id(BIOMASS_DP).flux))

MBR Weights 10:1:  13.402800000000015
SA growth: 0.0
DP growth: 1.3402800000000015
